In [6]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import tree
import matplotlib.pyplot as plt


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

kaggle/input/titanic/test.csv
kaggle/input/titanic/train.csv


In [7]:
train_data = pd.read_csv("kaggle/input/titanic/train.csv")
train_data.head()

test_data = pd.read_csv("kaggle/input/titanic/test.csv")
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [8]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import LabelEncoder  


device = torch.device('mps') if torch.backends.mps.is_available() else torch.device('cpu')


class Model(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(Model, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, output_dim),
            nn.ReLU(),
            nn.Linear(output_dim, 1),
        )
    
    def forward(self, x):
        out = self.model(x)
        return out

model = Model(5, 1024)
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [9]:
X_Train = train_data.drop(["Survived"], axis=1)
X_Train = X_Train.drop(["Name"], axis=1)
X_Train = X_Train.drop(["Ticket"], axis=1)
X_Train = X_Train.drop(["Cabin"], axis=1)
X_Train = X_Train.drop(["Embarked"], axis=1)
X_Train = X_Train.drop(["PassengerId"], axis=1)
X_Train = X_Train.drop(["Fare"], axis=1)
X_Train["Age"] = X_Train["Age"].fillna(X_Train["Age"].mean())

le = LabelEncoder()
X_Train["Sex"] = le.fit_transform(X_Train["Sex"])
y = train_data["Survived"]

X_Train.head()

,Pclass,Sex,Age,SibSp,Parch
0,3,1,22.0,1,0
1,1,0,38.0,1,0
2,3,0,26.0,0,0
3,1,0,35.0,1,0
4,3,1,35.0,0,0


In [10]:

epoch = 2000
X_Train = torch.from_numpy(X_Train.to_numpy()).float()
y = torch.from_numpy(y.to_numpy()).float()

train_data = DataLoader(TensorDataset(X_Train, y), batch_size=32, shuffle=True)

for n in range(epoch):
    for x, y_test in train_data:
        optimizer.zero_grad()
        y_pred = model(x)
        loss = nn.functional.mse_loss(y_pred, y_test)
        loss.backward()
        optimizer.step()

    print("Epoch: ", n, "Loss: ", loss.item())

Epoch:  0 Loss:  6.006331443786621
Epoch:  1 Loss:  132.20191955566406
Epoch:  2 Loss:  3.83335280418396
Epoch:  3 Loss:  4.471628665924072
Epoch:  4 Loss:  5.38202428817749
Epoch:  5 Loss:  4.138099670410156
Epoch:  6 Loss:  5.783787727355957
Epoch:  7 Loss:  2.867271900177002
Epoch:  8 Loss:  3.655320882797241
Epoch:  9 Loss:  3.214594841003418
Epoch:  10 Loss:  3.5138649940490723
Epoch:  11 Loss:  2.9866154193878174


/var/folders/s0/xcg0d2_s6tg_2v3r53qlxyl40000gn/T/ipykernel_1681/4023447541.py:11: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = nn.functional.mse_loss(y_pred, y_test)
/var/folders/s0/xcg0d2_s6tg_2v3r53qlxyl40000gn/T/ipykernel_1681/4023447541.py:11: UserWarning: Using a target size (torch.Size([27])) that is different to the input size (torch.Size([27, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = nn.functional.mse_loss(y_pred, y_test)


Epoch:  12 Loss:  1.3982625007629395
Epoch:  13 Loss:  3.7383921146392822
Epoch:  14 Loss:  3.4050655364990234
Epoch:  15 Loss:  1.7782527208328247
Epoch:  16 Loss:  1.8037501573562622
Epoch:  17 Loss:  2.703106164932251
Epoch:  18 Loss:  3.254760503768921
Epoch:  19 Loss:  1.4980885982513428
Epoch:  20 Loss:  2.4570152759552
Epoch:  21 Loss:  2.3002376556396484
Epoch:  22 Loss:  3.756526470184326
Epoch:  23 Loss:  2.1090757846832275
Epoch:  24 Loss:  1.9876052141189575
Epoch:  25 Loss:  2.7222049236297607
Epoch:  26 Loss:  1.8867124319076538
Epoch:  27 Loss:  2.308049440383911
Epoch:  28 Loss:  1.7390897274017334
Epoch:  29 Loss:  1.6795306205749512
Epoch:  30 Loss:  1.926452398300171
Epoch:  31 Loss:  1.572803020477295
Epoch:  32 Loss:  1.9039994478225708
Epoch:  33 Loss:  1.6884857416152954
Epoch:  34 Loss:  7.779623508453369
Epoch:  35 Loss:  2.1834909915924072
Epoch:  36 Loss:  2.0682272911071777
Epoch:  37 Loss:  1.9618486166000366
Epoch:  38 Loss:  1.1949812173843384
Epoch:  39 

In [ ]:
output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")